In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install medspacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.0/425.0 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 69.2 MB/s eta 0:00:00
  Created wheel for medspacy: filename=medspacy-1.3.1-py3-none-any.whl size=313644 sha256=aac56d67a3e4a646858829145566258d9330226be30d084ff90731700ce95ded
  Stored in directory: /root/.ca

In [ ]:
# prompt: change directory to current python notebooks on Google drive
import os
os.chdir('/content/drive/MyDrive/Courses/Ryerson/MRP/Dataset')

In [ ]:
import pandas as pd
import numpy as np
import medspacy
from sklearn.neighbors import KNeighborsClassifier as knn

In [ ]:
nlp = medspacy.load()

In [ ]:
train_datadf = pd.read_csv('drugLibTrain_raw.tsv', sep='\t', index_col=0)
train_labels = train_datadf['rating']
test_datadf = pd.read_csv('drugLibTest_raw.tsv', sep='\t', index_col=0)
test_labels = test_datadf['rating']

In [ ]:
train_datadf[train_datadf['sideEffectsReview'].isna()]

,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
1534,levitra,10,Highly Effective,No Side Effects,ed,The stuff works like a charm--fast and effective.,NaN,One pill 30 mins. before activity
2075,metformin,9,Highly Effective,No Side Effects,anti-aging prophilatic,At the recommendation of the Life Extension fo...,NaN,Metformin reduced my fasting blood glucose by ...
591,zestoretic,10,Highly Effective,No Side Effects,high blood pressure,After trying a variety of high blood pressure ...,NaN,I take two tablets once a day at breakfast.
2224,allegra,10,Highly Effective,No Side Effects,severe allergies,"Almost immediate relief of allergy symptoms, f...",NaN,I suffered from severe allergic reactions to a...
9,nasonex,8,Considerably Effective,No Side Effects,sinusitis,Relieved symptoms of itchy runny nose providin...,NaN,Use inhaler twice a day once in the morning an...
...,...,...,...,...,...,...,...,...
1169,nasarel,10,Highly Effective,No Side Effects,rhinitis,"Prior to using this spray, I had year round na...",NaN,I believe the recommended dose is two sprays p...
1874,doxycycline,6,Moderately Effective,No Side Effects,acne,Controlled acne breakouts to some extent when ...,NaN,Took Doxylin 50 mg once daily for 3 months. I...
3954,prilosec,9,Considerably Effective,No Side Effects,severe indigestion diagnosed as possible ulcer,"Although I was meant to take these daily, I fe...",NaN,Not sure I understand this request! I was pres...
2384,synthroid,9,Considerably Effective,No Side Effects,tiredness,"Less tired, no need for naps in the afternoon....",NaN,One pill daily in the morning one hour before ...


There are 75 rows with nan as values for side effects. We need to set this to None before retrieving tokens

In [ ]:
# train_datadf[train_datadf['sideEffectsReview'] ]

Clean the data

In [ ]:
# train_datadf['sideEffectsReview'] = train_datadf[train_datadf['sideEffectsReview'] ]
1327

In [ ]:
train_datadf.loc[train_datadf['sideEffectsReview'] == np.nan, 'sideEffectsReview'] = 'None'
train_datadf.loc[train_datadf['sideEffectsReview'] == 'nan', 'sideEffectsReview'] = 'None'
train_datadf.loc[train_datadf['sideEffectsReview'] == 'None.', 'sideEffectsReview'] = 'None'
train_datadf.loc[train_datadf['sideEffectsReview'] == 'none', 'sideEffectsReview'] = 'None'
train_datadf.loc[train_datadf['sideEffectsReview'] == 'no', 'sideEffectsReview'] = 'None'
train_datadf.loc[train_datadf['sideEffectsReview'] == '*', 'sideEffectsReview'] = 'None'
train_datadf.loc[train_datadf['sideEffectsReview'] == '..', 'sideEffectsReview'] = 'None'
train_datadf.loc[train_datadf['sideEffectsReview'].isna(), 'sideEffectsReview'] = 'None'
train_datadf.loc[train_datadf['sideEffectsReview'] == 'see above', 'sideEffectsReview'] = train_datadf['benefitsReview']
# no
# *
# ..

In [ ]:
test_datadf.loc[test_datadf['sideEffectsReview'] == np.nan, 'sideEffectsReview'] = 'None'
test_datadf.loc[test_datadf['sideEffectsReview'] == 'nan', 'sideEffectsReview'] = 'None'
test_datadf.loc[test_datadf['sideEffectsReview'] == 'None.', 'sideEffectsReview'] = 'None'
test_datadf.loc[test_datadf['sideEffectsReview'] == 'none', 'sideEffectsReview'] = 'None'
test_datadf.loc[test_datadf['sideEffectsReview'] == 'no', 'sideEffectsReview'] = 'None'
test_datadf.loc[test_datadf['sideEffectsReview'] == '*', 'sideEffectsReview'] = 'None'
test_datadf.loc[test_datadf['sideEffectsReview'] == '..', 'sideEffectsReview'] = 'None'
test_datadf.loc[test_datadf['sideEffectsReview'].isna(), 'sideEffectsReview'] = 'None'
test_datadf.loc[test_datadf['sideEffectsReview'] == 'see above', 'sideEffectsReview'] = test_datadf['benefitsReview']

**Labels set to ratings**

Using TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words="english", max_features=1000)
vectorizer.fit(train_datadf['sideEffectsReview'])

TfidfVectorizer(max_features=1000, stop_words='english')

Vecotrize the training and test dataset tokens

In [ ]:
train_array=vectorizer.transform(train_datadf['sideEffectsReview']).toarray()
train_array.shape

(3107, 1000)

In [ ]:
train_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
test_array= vectorizer.transform(test_datadf['sideEffectsReview']).toarray()

In [ ]:
test_array.shape

(1036, 1000)

In [ ]:
test_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
vectorizer.vocabulary_

{'cough': 184,
 'weakness': 965,
 'fatigue': 325,
 'heavy': 412,
 'cycle': 197,
 'cramps': 191,
 'hot': 424,
 'flashes': 341,
 'long': 523,
 'lasting': 489,
 'months': 585,
 'different': 231,
 'time': 897,
 'using': 940,
 'kind': 480,
 'unfortunately': 930,
 'constant': 178,
 'happy': 393,
 'results': 738,
 'bleeding': 106,
 'normal': 618,
 'constipation': 180,
 'dry': 269,
 'mouth': 592,
 'mild': 572,
 'dizziness': 243,
 'away': 82,
 'medication': 556,
 'stopped': 833,
 'days': 204,
 'felt': 334,
 'extremely': 311,
 'drive': 262,
 'med': 555,
 'extreme': 310,
 'ankle': 49,
 'feet': 332,
 'swelling': 858,
 'couldn': 186,
 'wear': 966,
 'low': 537,
 'sex': 774,
 'started': 822,
 'taking': 871,
 'drug': 267,
 'years': 995,
 'age': 39,
 'old': 639,
 'tired': 901,
 'hours': 426,
 'day': 203,
 'hard': 394,
 'concentrate': 169,
 'decreased': 210,
 'appetite': 61,
 'negative': 609,
 'affect': 36,
 'short': 779,
 'term': 879,
 'memory': 560,
 'really': 712,
 'don': 251,
 'effects': 284,
 'issu

Fit KNN Model for Ratings 1 - 10

In [ ]:
knn_model = knn(n_neighbors=10)
knn_model.fit(train_array, train_labels)
# knn_model.score(test_array, test_labels)

KNeighborsClassifier(n_neighbors=10)

Predict the test ratings

In [ ]:
# test_array = test_array.reshape(1, -1)
def predict(knn_model, predict_name, df, test_array):
  predictions = []
  for row in test_array:
    # print(row)
    pred = knn_model.predict([row])
    predictions.append(pred[0])
  df[predict_name] = predictions

In [ ]:
# predictions = []
# for row in test_array:
#   # print(row)
#   pred = knn_model.predict([row])
#   predictions.append(pred)
# test_datadf['predictions'] = predictions
# predict(df, test_array):
predict(knn_model, 'ratings_predictions', test_datadf, test_array)
  # break

In [ ]:
test_datadf.head(20)

,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview,ratings_predictions,sd_predictions,sideEffectsRatings
1366,biaxin,9,Considerably Effective,Mild Side Effects,sinus infection,The antibiotic may have destroyed bacteria cau...,"Some back pain, some nauseau.",Took the antibiotics for 14 days. Sinus infect...,1,[1],4
3724,lamictal,9,Highly Effective,Mild Side Effects,bipolar disorder,Lamictal stabilized my serious mood swings. On...,"Drowsiness, a bit of mental numbness. If you t...",Severe mood swings between hypomania and depre...,10,[5],4
3824,depakene,4,Moderately Effective,Severe Side Effects,bipolar disorder,Initial benefits were comparable to the brand ...,"Depakene has a very thin coating, which caused...",Depakote was prescribed to me by a Kaiser psyc...,10,[5],2
969,sarafem,10,Highly Effective,No Side Effects,bi-polar / anxiety,It controlls my mood swings. It helps me think...,I didnt really notice any side effects.,This drug may not be for everyone but its wond...,10,[5],5
696,accutane,10,Highly Effective,Mild Side Effects,nodular acne,Within one week of treatment superficial acne ...,Side effects included moderate to severe dry s...,Drug was taken in gelatin tablet at 0.5 mg per...,10,[5],4
1380,biaxin,2,Marginally Effective,No Side Effects,sinus infection,By the end of the 10-day treatment I felt bett...,I felt no significant side effects - perhaps s...,Basically the treatment did not seem to work. ...,10,[5],5
45,carbamazepine,8,Considerably Effective,Moderate Side Effects,seizure,reduction in seizures reduction in seizures re...,tired/sleepy very tired sleep and tired very t...,took it for seizure took pills drank with wate...,10,[4],3
1939,ultram-er,10,Highly Effective,Mild Side Effects,cervical disk degeneration and lower back pain,Ive been taking Tramadol for 2 weeks now. Ive ...,I have had no side effects so far. I hope it s...,"Treating for neck, shoulder, arms, lower back,...",10,[5],4
2576,klonopin,10,Highly Effective,No Side Effects,panic disorder,I immediately stopped having panic attacks. I...,I experienced no side effects. I was not tire...,I started both klonopin and prozac together. ...,9,[5],5
1093,effexor,1,Marginally Effective,Extremely Severe Side Effects,depression,the presumed benefits were to help with a seve...,here we go.the initial effect would be dry mou...,family doctor initially prescribed wellbutin b...,10,[5],1


Check the accuracy for the prediction

Using sideEffects as the label

In [ ]:
# train_datadf.loc[train_datadf['sideEffects'] == 'Mild Side Effects', 'sideEffectsRatings'] = 'None'
# test_datadf.loc[test_datadf['sideEffects'] == np.nan, 'sideEffectsReview'] = 'None'

In [ ]:
train_datadf['sideEffectsRatings'] = train_datadf['sideEffects'].replace({'No Side Effects': 5,
    'Mild Side Effects': 4,
    'Moderate Side Effects': 3,
    'Severe Side Effects': 2,
    'Extremely Severe Side Effects': 1})

In [ ]:
test_datadf['sideEffectsRatings'] = test_datadf['sideEffects'].replace({'No Side Effects': 5,
    'Mild Side Effects': 4,
    'Moderate Side Effects': 3,
    'Severe Side Effects': 2,
    'Extremely Severe Side Effects': 1})

In [ ]:
train_datadf.head()

,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview,sideEffectsRatings
2202,enalapril,4,Highly Effective,Mild Side Effects,management of congestive heart failure,slowed the progression of left ventricular dys...,"cough, hypotension , proteinuria, impotence , ...","monitor blood pressure , weight and asses for ...",4
3117,ortho-tri-cyclen,1,Highly Effective,Severe Side Effects,birth prevention,Although this type of birth control has more c...,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...","I Hate This Birth Control, I Would Not Suggest...",2
1146,ponstel,10,Highly Effective,No Side Effects,menstrual cramps,I was used to having cramps so badly that they...,Heavier bleeding and clotting than normal.,I took 2 pills at the onset of my menstrual cr...,5
3947,prilosec,3,Marginally Effective,Mild Side Effects,acid reflux,The acid reflux went away for a few months aft...,"Constipation, dry mouth and some mild dizzines...",I was given Prilosec prescription at a dose of...,4
1951,lyrica,2,Marginally Effective,Severe Side Effects,fibromyalgia,I think that the Lyrica was starting to help w...,I felt extremely drugged and dopey. Could not...,See above,2


In [ ]:
knn_sd_model = knn(n_neighbors=5)
knn_sd_model.fit(train_array, train_datadf['sideEffectsRatings'])

KNeighborsClassifier()

Predict Side Effect Ratings

In [ ]:
# knn_sd_model.
predict(knn_sd_model, 'sd_predictions', test_datadf, test_array)

In [ ]:
test_datadf.head(20)

,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview,ratings_predictions,sd_predictions,sideEffectsRatings
1366,biaxin,9,Considerably Effective,Mild Side Effects,sinus infection,The antibiotic may have destroyed bacteria cau...,"Some back pain, some nauseau.",Took the antibiotics for 14 days. Sinus infect...,1,1,4
3724,lamictal,9,Highly Effective,Mild Side Effects,bipolar disorder,Lamictal stabilized my serious mood swings. On...,"Drowsiness, a bit of mental numbness. If you t...",Severe mood swings between hypomania and depre...,10,5,4
3824,depakene,4,Moderately Effective,Severe Side Effects,bipolar disorder,Initial benefits were comparable to the brand ...,"Depakene has a very thin coating, which caused...",Depakote was prescribed to me by a Kaiser psyc...,10,5,2
969,sarafem,10,Highly Effective,No Side Effects,bi-polar / anxiety,It controlls my mood swings. It helps me think...,I didnt really notice any side effects.,This drug may not be for everyone but its wond...,10,5,5
696,accutane,10,Highly Effective,Mild Side Effects,nodular acne,Within one week of treatment superficial acne ...,Side effects included moderate to severe dry s...,Drug was taken in gelatin tablet at 0.5 mg per...,10,5,4
1380,biaxin,2,Marginally Effective,No Side Effects,sinus infection,By the end of the 10-day treatment I felt bett...,I felt no significant side effects - perhaps s...,Basically the treatment did not seem to work. ...,10,5,5
45,carbamazepine,8,Considerably Effective,Moderate Side Effects,seizure,reduction in seizures reduction in seizures re...,tired/sleepy very tired sleep and tired very t...,took it for seizure took pills drank with wate...,10,4,3
1939,ultram-er,10,Highly Effective,Mild Side Effects,cervical disk degeneration and lower back pain,Ive been taking Tramadol for 2 weeks now. Ive ...,I have had no side effects so far. I hope it s...,"Treating for neck, shoulder, arms, lower back,...",10,5,4
2576,klonopin,10,Highly Effective,No Side Effects,panic disorder,I immediately stopped having panic attacks. I...,I experienced no side effects. I was not tire...,I started both klonopin and prozac together. ...,9,5,5
1093,effexor,1,Marginally Effective,Extremely Severe Side Effects,depression,the presumed benefits were to help with a seve...,here we go.the initial effect would be dry mou...,family doctor initially prescribed wellbutin b...,10,5,1


Classification Report

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(test_datadf['sideEffectsRatings'], test_datadf['sd_predictions'])
print("Classification Report:\n")
print(report)

Classification Report:

              precision    recall  f1-score   support

           1       0.50      0.01      0.02        80
           2       0.29      0.02      0.03       122
           3       0.38      0.04      0.07       236
           4       0.61      0.21      0.31       330
           5       0.28      0.94      0.44       268

    accuracy                           0.32      1036
   macro avg       0.41      0.24      0.17      1036
weighted avg       0.42      0.32      0.23      1036



In [ ]:
# train_datadf['side_effects_tokens'] = [nlp(sd) for sd in train_datadf['sideEffectsReview'][:50]]
# train_datadf = [nlp(sd) for sd in train_datadf['sideEffectsReview'][:50]]
tokens = []
for sd in train_datadf['sideEffectsReview'][:50]:
  print(sd)
  print('\n')
  sentence = nlp(sd)
  tokens.append([token.text for token in sentence])

cough, hypotension , proteinuria, impotence , renal failure , angina pectoris , tachycardia , eosinophilic pneumonitis, tastes disturbances , anusease anorecia , weakness fatigue insominca weakness


Heavy Cycle, Cramps, Hot Flashes, Fatigue, Long Lasting Cycles. It's only been 5 1/2 months, but i'm concidering changing to a different bc. This is my first time using any kind of bc, unfortunately due to the constant hassel, i'm not happy with the results.


Heavier bleeding and clotting than normal.


Constipation, dry mouth and some mild dizziness that would go away after medication was stopped for a few days.


I felt extremely drugged and dopey.  Could not drive at all while on this med.  Also had extreme ankle and feet swelling and couldn't even wear shoes.


Low sex drive, before i started taking this drug i was 19 years of age and full of "enthousiasm". Now engaged and 21 years old, my sex drive is very low, my wife and I are very unsatisfied. I am also very tired even during peak

In [ ]:
tokens

[['cough',
  ',',
  'hypotension',
  ',',
  'proteinuria',
  ',',
  'impotence',
  ',',
  'renal',
  'failure',
  ',',
  'angina',
  'pectoris',
  ',',
  'tachycardia',
  ',',
  'eosinophilic',
  'pneumonitis',
  ',',
  'tastes',
  'disturbances',
  ',',
  'anusease',
  'anorecia',
  ',',
  'weakness',
  'fatigue',
  'insominca',
  'weakness'],
 ['Heavy',
  'Cycle',
  ',',
  'Cramps',
  ',',
  'Hot',
  'Flashes',
  ',',
  'Fatigue',
  ',',
  'Long',
  'Lasting',
  'Cycles',
  '.',
  'It',
  "'s",
  'only',
  'been',
  '5',
  '1',
  '/',
  '2',
  'months',
  ',',
  'but',
  'i',
  "'m",
  'concidering',
  'changing',
  'to',
  'a',
  'different',
  'bc',
  '.',
  'This',
  'is',
  'my',
  'first',
  'time',
  'using',
  'any',
  'kind',
  'of',
  'bc',
  ',',
  'unfortunately',
  'due',
  'to',
  'the',
  'constant',
  'hassel',
  ',',
  'i',
  "'m",
  'not',
  'happy',
  'with',
  'the',
  'results',
  '.'],
 ['Heavier', 'bleeding', 'and', 'clotting', 'than', 'normal', '.'],
 ['Constip

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=tokens,
                 vector_size=100,
                 window= 5,
                 min_count=1,
                 workers=4,
                 epochs=10)

In [ ]:
np.mean([model.wv[e] for e in a_sentence if e in model.wv.key_to_index], axis=0)

In [ ]:
model.wv.most_similar("cramps")

[('Lightheadedness', 0.33527883887290955),
 ('switched', 0.31532666087150574),
 ('eventually', 0.27015429735183716),
 ('feathery', 0.24063614010810852),
 ('anusease', 0.23972435295581818),
 ('placing', 0.23513995110988617),
 ('cream', 0.22801125049591064),
 ('ruined', 0.22527816891670227),
 ('example', 0.22171340882778168),
 ('excessive', 0.21932753920555115)]

In [ ]:
model.wv["cramps"]

array([ 0.00679924, -0.0086197 , -0.00700092, -0.00593746,  0.00049569,
       -0.00673912,  0.00678229,  0.00509763, -0.00466458,  0.00775377,
        0.00751887, -0.00597818,  0.00427842, -0.00876187,  0.0045871 ,
       -0.00934277, -0.00522621, -0.0002076 , -0.00343829,  0.00502155,
        0.00156868,  0.00681098, -0.00035432, -0.00024526, -0.00075051,
        0.00336896,  0.00553459,  0.00914703, -0.00228522,  0.00165331,
        0.0036322 , -0.00502074,  0.00728174,  0.00624336, -0.00836143,
       -0.00651566, -0.00623995, -0.00707217,  0.00897646,  0.00512942,
       -0.00322153, -0.00553284, -0.00073325,  0.00193267, -0.00764836,
       -0.00038432,  0.00578206, -0.00012485,  0.00343556, -0.00493608,
       -0.00048067, -0.00362032,  0.00501123,  0.00649955, -0.00772433,
        0.00028483, -0.00445071, -0.00872552, -0.00846368,  0.00779358,
       -0.00208933,  0.00781609, -0.00211505, -0.00379296,  0.00297857,
       -0.00381287, -0.00451913,  0.0087493 ,  0.00158907, -0.00

In [ ]:

clf=knn()
clf.fit(train_emb, train_labels)
clf.score(test_emb, test_labels)